In [3]:
import numpy as np 
import pandas as pd 
import tqdm
import string 
from sklearn.svm import LinearSVC 
from sklearn.calibration import CalibratedClassifierCV 

from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes

In [6]:
import glob
import pandas as pd
from tensorflow import keras
import numpy as np
import os 
from sklearn.model_selection import StratifiedKFold, KFold
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, BatchNormalization, concatenate, Input, ConvLSTM2D, Reshape, Conv3D, Flatten, LSTM, GRU, Dense,Dropout, Add
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalMaxPool1D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Sequential, load_model
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import re 
from catboost import CatBoostClassifier

In [7]:
train = pd.read_csv('./storage/fintech_nlp/lgbm_train_df.csv') 
test = pd.read_csv('./storage/fintech_nlp/lgbm_test_df.csv')

In [8]:
x_train = train.drop(['n_id','info'], axis = 1) 
y_train = train['info'] 

In [9]:
x_test = test.drop(['n_id','id'], axis=1) 

In [10]:
x_train.shape, y_train.shape, x_test.shape

((118745, 67), (118745,), (142565, 67))

In [11]:
text_cols = ['title','content','title_tokenized','content_tokenized']

In [ ]:
k = 5 
models = [] 
kfold = StratifiedKFold(n_splits = k, shuffle = True, random_state = 990101) 
for n_fold, (train_idx, val_idx) in enumerate(kfold.split(x_train, y_train)): 
    train_x, val_x = x_train.loc[train_idx,:], x_train.loc[val_idx,:]
    train_y, val_y = y_train.loc[train_idx], y_train.loc[val_idx] 
    
    model = CatBoostClassifier(iterations = 5000, 
                               learning_rate = 0.01, 
                               l2_leaf_reg = 3.5,
                               depth = 8, 
                               loss_function = 'Logloss',
                               eval_metric = 'Logloss',
                               use_best_model = True, 
                               task_type = 'GPU',
                               random_seed = 960418)  
    
    
    model.fit(train_x, train_y, eval_set = (val_x, val_y), text_features = text_cols) 
    models.append(model)

0:	learn: 0.6689592	test: 0.6685823	best: 0.6685823 (0)	total: 23.2ms	remaining: 1m 55s
1:	learn: 0.6444832	test: 0.6439304	best: 0.6439304 (1)	total: 46.5ms	remaining: 1m 56s
2:	learn: 0.6242329	test: 0.6232733	best: 0.6232733 (2)	total: 72.7ms	remaining: 2m 1s
3:	learn: 0.6039025	test: 0.6026070	best: 0.6026070 (3)	total: 95ms	remaining: 1m 58s
4:	learn: 0.5830719	test: 0.5814810	best: 0.5814810 (4)	total: 117ms	remaining: 1m 56s
5:	learn: 0.5641886	test: 0.5620821	best: 0.5620821 (5)	total: 139ms	remaining: 1m 55s
6:	learn: 0.5429981	test: 0.5405743	best: 0.5405743 (6)	total: 159ms	remaining: 1m 53s
7:	learn: 0.5265575	test: 0.5239258	best: 0.5239258 (7)	total: 181ms	remaining: 1m 52s
8:	learn: 0.5102234	test: 0.5073773	best: 0.5073773 (8)	total: 205ms	remaining: 1m 53s
9:	learn: 0.4958921	test: 0.4929619	best: 0.4929619 (9)	total: 231ms	remaining: 1m 55s
10:	learn: 0.4803943	test: 0.4772546	best: 0.4772546 (10)	total: 252ms	remaining: 1m 54s
11:	learn: 0.4646514	test: 0.4612163	bes

In [ ]:
pred1 = models[0].predict_proba(x_test) 
pred2 = models[1].predict_proba(x_test) 
pred3 = models[2].predict_proba(x_test) 
pred4 = models[3].predict_proba(x_test) 
pred5 = models[4].predict_proba(x_test)